# Разпознаване на именувани обекти (NER)

Този ноутбук е част от [Учебната програма за начинаещи в изкуствения интелект](http://aka.ms/ai-beginners).

В този пример ще научим как да обучим модел за разпознаване на именувани обекти (NER) върху [Анотиран корпус за разпознаване на именувани обекти](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus) от Kaggle. Преди да продължите, моля, изтеглете файла [ner_dataset.csv](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus?resource=download&select=ner_dataset.csv) в текущата директория.


In [62]:
import pandas as pd
from tensorflow import keras
import numpy as np

## Подготовка на набора от данни

Ще започнем, като заредим набора от данни в dataframe. Ако искате да научите повече за използването на Pandas, посетете [урок за обработка на данни](https://github.com/microsoft/Data-Science-For-Beginners/tree/main/2-Working-With-Data/07-python) в нашия [Data Science for Beginners](http://aka.ms/datascience-beginners).


In [3]:
df = pd.read_csv('ner_dataset.csv',encoding='unicode-escape')
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


Нека получим уникални тагове и създадем речници за търсене, които можем да използваме за преобразуване на таговете в номера на класове:


In [4]:
tags = df.Tag.unique()
tags

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [8]:
id2tag = dict(enumerate(tags))
tag2id = { v : k for k,v in id2tag.items() }

id2tag[0]

'O'

Сега трябва да направим същото с речника. За простота ще създадем речник, без да вземаме предвид честотата на думите; в реалния живот може да искате да използвате Keras vectorizer и да ограничите броя на думите.


In [14]:
vocab = set(df['Word'].apply(lambda x: x.lower()))
id2word = { i+1 : v for i,v in enumerate(vocab) }
id2word[0] = '<UNK>'
vocab.add('<UNK>')
word2id = { v : k for k,v in id2word.items() }

Трябва да създадем набор от данни с изречения за обучение. Нека преминем през оригиналния набор от данни и разделим всички отделни изречения на `X` (списъци от думи) и `Y` (списък от токени):


In [41]:
X,Y = [],[]
s,t = [],[]
for i,row in df[['Sentence #','Word','Tag']].iterrows():
    if pd.isna(row['Sentence #']):
        s.append(row['Word'])
        t.append(row['Tag'])
    else:
        if len(s)>0:
            X.append(s)
            Y.append(t)
        s,t = [row['Word']],[row['Tag']]
X.append(s)
Y.append(t)


In [93]:
def vectorize(seq):
    return [word2id[x.lower()] for x in seq]

def tagify(seq):
    return [tag2id[x] for x in seq]

Xv = list(map(vectorize,X))
Yv = list(map(tagify,Y))

Xv[0], Yv[0]

([10386,
  23515,
  4134,
  29620,
  7954,
  13583,
  21193,
  12222,
  27322,
  18258,
  5815,
  15880,
  5355,
  25242,
  31327,
  18258,
  27067,
  23515,
  26444,
  14412,
  358,
  26551,
  5011,
  30558],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0])

За простота ще допълним всички изречения с 0 токени до максималната дължина. В реалния живот може да искаме да използваме по-умна стратегия и да допълваме последователности само в рамките на един минипакет.


In [51]:
X_data = keras.preprocessing.sequence.pad_sequences(Xv,padding='post')
Y_data = keras.preprocessing.sequence.pad_sequences(Yv,padding='post')

## Дефиниране на мрежа за класификация на токени

Ще използваме двуслойна двупосочна LSTM мрежа за класификация на токени. За да приложим плътен класификатор към всеки от изходите на последния LSTM слой, ще използваме конструкцията `TimeDistributed`, която репликира същия плътен слой към всеки от изходите на LSTM на всяка стъпка:


In [94]:
maxlen = X_data.shape[1]
vocab_size = len(vocab)
num_tags = len(tags)
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, 300, input_length=maxlen),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.TimeDistributed(keras.layers.Dense(num_tags, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 104, 300)          9545400   
                                                                 
 bidirectional_6 (Bidirectio  (None, 104, 200)         320800    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 104, 200)         240800    
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, 104, 17)          3417      
 tributed)                                                       
                                                                 
Total params: 10,110,417
Trainable params: 10,110,417
Non-trainable params: 0
__________________________________________

Имайте предвид, че тук изрично задаваме `maxlen` за нашия набор от данни - ако искаме мрежата да може да обработва последователности с променлива дължина, трябва да бъдем малко по-умни при дефинирането на мрежата.

Сега нека обучим модела. За по-бърза работа ще обучаваме само за една епоха, но можете да опитате да обучавате за по-дълго време. Също така, може да искате да отделите част от набора от данни като тренировъчен набор, за да наблюдавате точността на валидацията.


In [57]:
model.fit(X_data,Y_data)

1499/1499 [==============================] - 740s 488ms/step - loss: 0.0667 - acc: 0.9841


## Тестване на резултата

Нека сега видим как работи нашият модел за разпознаване на обекти върху примерно изречение:


In [91]:
sent = 'John Smith went to Paris to attend a conference in cancer development institute'
words = sent.lower().split()
v = keras.preprocessing.sequence.pad_sequences([[word2id[x] for x in words]],padding='post',maxlen=maxlen)
res = model(v)[0]

In [92]:
r = np.argmax(res.numpy(),axis=1)
for i,w in zip(r,words):
    print(f"{w} -> {id2tag[i]}")

john -> B-per
smith -> I-per
went -> O
to -> O
paris -> B-geo
to -> O
attend -> O
a -> O
conference -> O
in -> O
cancer -> B-org
development -> I-org
institute -> I-org


## Основни изводи

Дори прост LSTM модел показва задоволителни резултати при NER. Въпреки това, за да постигнете значително по-добри резултати, може да искате да използвате големи предварително обучени езикови модели като BERT. Обучението на BERT за NER с помощта на библиотеката Huggingface Transformers е описано [тук](https://huggingface.co/course/chapter7/2?fw=pt).



---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Ние не носим отговорност за недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
